## Realtime twitter data scraping using Tweepy and SNSscrape

In [196]:
#There are packages that might require installation
# !pip install twitterscraper
# !pip install tweepy

In [197]:
#Load packages from tweepy
import tweepy as tp

#Load packaes from twitterscraper
import twitterscraper
from twitterscraper import query_tweets

#Load package from snscrape to scrape twitters frontend
import snscrape.modules.twitter as sntwitter

#NLP Modules
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras import utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

import gmplot

#Load additional packages
import time
import numpy as np
import pandas as pd
import requests
import re
import itertools
from collections import Counter
import sys
from retry import retry

Insert your twitter api keys below as strings in order for the rest of the book to run

In [198]:
#Personal APIs (NEED TO CONVERT THIS TO LOCAL VARIABLES BEFORE HANDING IN)
#Add API keys here
consumer_key = 'Be42F1ILEuSM8t9UNNwywpNtw'
consumer_secret = '0B8dI0bwTBPIpWGU0lT7LamwVHK2mIC2k0E1ErXsHCvEKERnxG'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEDjMgEAAAAAo%2BG1VrFDLs5HxCECRlfgvNqg40Q%3Dr0MdX6JhXhimozIjUF6DiWkAFNEeSa1Vn263J5yu2mF2ZkDOnj'
access_token = '983900877919588352-ah6LqQhYEh5QjRBZ9TiBaSsFceOvaRG'
access_token_secret = 'kcLhKIaRK8FL4Dh6Tmw9xFiLaTpLj9Zesxh53mrABSwS5'

Authorize api keys in order to use API

In [199]:
#Authorization using api keys
auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tp.API(auth)

Import the top 100 twitter accounts posting natural disaster related news and append disasterchart and Distaster_Update to the list to make a total of 102 accounts to scrape for status updates

In [200]:
#Import the top 100 twitter natural disaster reporting accounts
df_100 = pd.read_csv('data/top100_twittwer_disaster_accounts.csv')
df_100['Account Handle'] = df_100['Account Handle'].str.replace('\W', '')
#Convert account handles to series
s_100 = df_100['Account Handle']
s_add = pd.Series(['disasterschart','Disaster_Update'])
s_102 = s_100.append(s_add)

##Initialize a list
online = []

#Loop to check whether account names are still active or not
for screen_name in s_102:
    try:
        user = api.get_user(screen_name)
        online.append(user.screen_name)
    except:
        pass

#Final series of active account names
s = pd.Series(online)

### Define functions to scrape twitter

Tweepy scrape function is used to scrape the top 102 account statuses for updates

In [201]:
#Initilization
array = [[]]

In [202]:
#Pass in any series list of twitter account names
@retry(tries=10,delay=2,backoff=4,max_delay=42)
def get_top_tweets(s,num_posts):
    for screen_name in s:
        try:
            #Initialize
            hashtag_list = []
            data = []
            #Define user
            user = api.get_user(screen_name)
            #Only pull the last __ number of tweets from each account 
            for tweet in api.user_timeline(screen_name = screen_name, count = num_posts):
                data.append(f'{screen_name}')
                data.append(tweet.created_at) 
                data.append(tweet.text)
                if len(tweet.entities.get('hashtags'))>0:
                    ht = [tweet.entities.get('hashtags')[x]['text'] for x in range(0,len(tweet.entities.get('hashtags')))]
                    hashtag_list.append(ht)
                data.append(hashtag_list)
                array.append(data)
                data = []
                hashtag_list = []

        except tp.TweepError:
            time.sleep(60*15)
            continue

Define a fucntion to convert list items to a single string joined by delimeter

In [203]:
#Define a function to convert list items to a single string joined by semicolon 
def list_to_string(x):
    lis = x
    string = ''
    string = ";".join(lis)
    return string

Define a funtion to search the dataframe containing tweets and pull out the unique hashtags and count their mentions

In [204]:
def hts_extract(df):
    #Convert hashtags columns to a list
    hashtags = df['hashtags'].tolist()

    #Initialize two empty lists for temp storage
    hts = []
    ht = []
    ht_final = []

    #Iterate through and remove blank hashtags
    for x in hashtags:
        if x == '':
            pass
        else:
            hts.append(x)

    #Iterate through and split out multiple hastags into additional list elements
    for x in hts:
        if ';' in x:
            el = x.split(';')
            ht = ht + el
        else:
            ht.append(x)

    for item in ht:
        ht_final.append(item.lower())
    
    #Count the number of times a hastag was mentioned, store this in a dictionary along with the keyword 
    #Sort the dictionary from highest to lowest values
    top_hts = dict(sorted(Counter(ht_final).items(), key=lambda item: item[1],reverse=True))
    return top_hts

Define a function to get relevant tweets with adjustable parameters of number of queries, start date and end date

In [205]:
@retry(tries=10,delay=2,backoff=4,max_delay=42)
def get_relevant_tweets(num_queries, start_date, end_date):
    
    #Intitilize empty list 
    tweets_list = [] 

    #For loop to go through each of the relevant keyword 
    for word in set_key_words:

        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{word} since:{start_date} until:{end_date}').get_items()):
            if i>num_queries:
                break
            tweets_list.append([tweet.username,tweet.date,tweet.content])
    
    return tweets_list

Function to pre-process tweet text, toeknize, lematize and remove stopwords

In [206]:
def process_tweet(df):
    tokenizer = RegexpTokenizer('\w+|\$[\.]\S+')
    lem = WordNetLemmatizer()
    STOPWORDS = set(stopwords.words('english'))
    
    df.rename(columns={'text':'tweet'},inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True)
    df['processed_tweets'] = df['tweet'].apply(tokenizer.tokenize)
    df['processed_tweets'] = df['processed_tweets'].apply(lambda row: list([lem.lemmatize(i) for i in row]))
    df['processed_tweets'] = df['processed_tweets'].apply(lambda x:[i for i in x if i not in STOPWORDS] )
    
    tokenizer = Tokenizer(num_words= 5000)
    tokenizer.fit_on_texts(df.processed_tweets.tolist())
    X = tokenizer.texts_to_sequences(df.processed_tweets.tolist())
    X = pad_sequences(X, maxlen = df.processed_tweets.str.len().max(), padding = 'post')
    return X

Define varaible for list of Keywords related to our target label tags coming out of the word2vec model

In [207]:
#Keywords related to our target label tags coming out of the word2vec model

key_words = ['flooding', 'mudslides', 'droughts', 'landslides', 'flood', 'storms', 'wildfires',
             'earthquakes', 'tornadoes','floodwaters', 'typhoons', 'disasters','tsunami', 'tsunamis', 
             'earthquake', 'explosions', 'exploded', 'explodes', 'blast', 'exploding', 'explode', 'bomb', 
             'detonation', 'implosion', 'earthquake', 'detonations', 'catastrophe', 'burst', 'fires', 'blasts',
             'quake', 'tsunami', 'earthquakes', 'aftershocks', 'quakes','floods', 'temblor', 'disaster', 
             'devastation', 'tsunamis', 'landslides', 'disasters', 'magnitude', 'mudslides', 'flood',
             'hurrican', 'storm', 'hurricanes', 'katrina', 'storms', 'tornado', 'cyclone', 'tsunami', 'typhoon',
             'disaster', 'landfall', 'earthquake', 'flood', 'huricane', 'tornadoes', 'tornadoes', 'tornados',
             'storm', 'thunderstorm', 'hurricane', 'tsunami', 'cyclone', 'storms', 'typhoon', 'floods',
             'snowstorm', 'tornadic', 'flood', 'earthquake', 'waterspout', 'bombings', 'bombed', 'bomb',
             'airstrikes', 'bomber', 'bombers', 'bombs', 'terrorist', 'airstrike', 'firebombing', 'attack', 
             'hijackings', 'attacks', 'bombardment', 'bombardments', 'mudslides', 'landslides', 'landslide', 
             'rainstorms', 'sinkhole', 'mudflows', 'earthquake', 'rockslide', 'snowstorm', 'kahlua', 'temblor',
             'mudflow', 'rainstorm', 'blizzards', 'snowstorms', 'snowpocalypse', 'snowstorm', 'snowstorms', 
             'snowmageddon', 'snowicane', 'rainstorm', 'squalls', 'snowtober', 'thundersnow', 'whiteout',
             'snowfall', 'pogies', 'windchills', 'blizzards', 'bostonist']
    
#Isolate unique keywords
set_key_words = set(key_words)

In [208]:
def map_preds(df):
    df['pred_label'] = df['predictions'].map({0: "bombing", 1: "earthquake", 2 : "explosion", 3: "floods" ,
                                              4: "hurricane" ,5: "mudslide" ,6: "neutral" ,7:"noreaster" ,
                                              8:"tornado", 9: "wildfire"})
    
def threshold_counts(df, threshold):
    counts = df.pred_label.value_counts()
    labels = df.pred_label.unique()
    disaster_counts = list(zip( labels, counts))
    flagged_disaster = []
    [flagged_disaster.append([f'There are {value} mentions of {key}']) for key, value in disaster_counts if value >= threshold]
    return flagged_disaster

In [209]:
cities = pd.read_csv('https://raw.git.generalassemb.ly/noahszuckerman/project-5-natty-ds/master/data/worldcities.csv?token=AAAH7Z45CMEZ3KD2BQCU23LAGU4UQ', encoding = 'latin')
cities.drop(columns = ['city_ascii', 'iso2', 'iso3', 'admin_name', 'capital',
       'population', 'id'], axis = 1, inplace = True)

# Joining the coordinates to create a tuple.
select_coords = list(zip(cities['lat'], cities['lng']))

#Filtering out cities in Canada and the United States for demonstration purposes.
#Additional cities will be included in production. 
select_cities = cities.loc[(cities['country'] == 'United States') | (cities['country'] == 'Canada')]
select_cities = select_cities['city'].tolist()
select_cities = [x.lower() for x in select_cities]
city_dict = dict(zip(select_cities,select_coords))


def map_location(df):
    location = []
    array = [[]]

    for tweet in df['processed_tweets']:

        for word in tweet:
            try:
                city_dict[word]
                location.append(tweet)
                location.append(word)
                array.append(location)
                location = []


            except:
                location.append(tweet)
                location.append('No Location Available')
                array.append(location)
                location = []

    location_df = pd.DataFrame(array, columns = ['tweet', 'location'])
    location_df.drop([0], inplace = True)

    location_df['tweet'] =  location_df['tweet'].apply(' '.join)
    location_df.drop_duplicates(inplace = True)
    return location_df


### Load model

In [210]:
model = load_model('Word2Vec_LSTM_model.h5')

## Main workflow

In [212]:
x = 1

while x> 0:
    
    #Scrape the top accounts last 10 posts
    get_top_tweets(s,10) #10 can be adjusted up or down. Any duplciates will be removed in the following transformations

    #Create a dataframe from array
    df = pd.DataFrame(array,columns=['screen_name','created_at','tweet','hashtags'])
    df.dropna(inplace = True)

    #Flatten out the hastags column for extraction
    df['hashtags'] = [list(itertools.chain.from_iterable(x)) for x in df['hashtags']]
    df['hashtags'] = df['hashtags'].apply(list_to_string)

    #Remove any dduplicate values in case accounts have not posted anything new
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True,inplace=True)
    
    #Call function to build sorted dictionary of top hashtag mentions
    top_hashtags_dic = hts_extract(df)
    
    top_hashtags_dic_filt = {k: v for k, v in top_hashtags_dic.items() if v > 5}
    
    for hr in range(0,25):
        
        #Reset tweet list to empty
        tweets_list = []
        
        #Call the get relevant tweets function to query all twitter posts for content matching our word2vec keywords
        tweets_list = get_relevant_tweets(num_queries=12, start_date = '2021-2-14', end_date = '2021-2-15')
        
        #Put the tweets list pull into a dataframe
        df2 = pd.DataFrame(tweets_list, columns=['screen_name','created_at','tweet'])

        #Remove the 'hashtags column' in the first dataset of tweets
        df1 = df[['screen_name','created_at','tweet']]

        #Combine both datasets
        df_ex = df1.append(df2)
        
        #Remove any dduplicate values in case accounts have not posted anything new
        df_ex.drop_duplicates(inplace=True)
        df_ex.reset_index(drop=True,inplace=True)
        
        #Pre-prcoess tweet text for model predictions
        
        df_ex = df_ex.assign(id=(df_ex['tweet'] + '_' + df_ex['screen_name']).astype('category').cat.codes)
        df_ex.id = df_ex.id.astype(str)
        df_ex['tweet'] = df_ex['tweet'] + ' 00' + df_ex['id']

        X = process_tweet(df_ex)

        df_ex['predictions'] = model.predict_classes(X)
        map_preds(df_ex)
        flagged_mentions = threshold_counts(df_ex, 10)  
        trial = map_location(df_ex)


        trial['id'] = [i[-4:] for i in trial['tweet']]

        df_ex['id'] = ' 00' + df_ex['id']
        df_ex['id'] = [i[-4:] for i in df_ex['id']]

        final_df = pd.merge(df_ex, trial, how = 'left', on = 'id')

        location_groups = (final_df.groupby(['pred_label', 'location']).size())


        
        #Display results
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S",t) 
        sys.stdout.write(f' \r| Time: {current_time} | {top_hashtags_dic_filt} | MODEL PREDICTIONS: | {flagged_mentions},                                                    {location_groups} ')
        sys.stdout.flush()
        
        time.sleep(60*60)
        
        if hr == 24:
            break
    
    
    t_now = time.asctime( time.localtime(time.time()))
    
    #Export the dataset as csv
    final_df.to_csv(f'data/predictions_{t_now}.csv',index=False)

/Users/mathea/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


| Time: 19:37:12 | {'covid19': 48, 'earthquake': 17, 'resilient': 10, 'cities': 10, 'iati': 6} | MODEL PREDICTIONS: | [['There are 675 mentions of hurricane'], ['There are 291 mentions of mudslide'], ['There are 226 mentions of earthquake'], ['There are 201 mentions of neutral'], ['There are 189 mentions of tornado'], ['There are 95 mentions of bombing'], ['There are 64 mentions of noreaster'], ['There are 64 mentions of floods'], ['There are 29 mentions of wildfire'], ['There are 28 mentions of explosion']],                                                    pred_label  location             
bombing     No Location Available    29 
            friendly                 1  
            pace                     1  
            page                     1  
earthquake  No Location Available    201
                                    ... 
wildfire    middle                   1  
            mission                  1  
            ontario                  1  
            union              

KeyboardInterrupt: 